In [2]:
from omegaconf import OmegaConf
from datasets import load_data
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from architectures import load_architecture

config = OmegaConf.load("./configs/default_config_linearprobe50.yaml")

backbones=(
  'convnext_tiny', 'robust_convnext_tiny', 'convnext_tiny.fb_in22k', 
  'deit_small_patch16_224.fb_in1k', 'robust_deit_small_patch16_224',
  'convnext_base', 'convnext_base.fb_in22k', 'robust_convnext_base', 
  'convnext_base.clip_laion2b', 'convnext_base.clip_laion2b_augreg',
  'vit_base_patch16_224.augreg_in1k', 'vit_base_patch16_224.augreg_in21k',
  'vit_base_patch16_224.dino', 'vit_base_patch16_224.mae', 'vit_base_patch16_224.orig_in21k',
  'vit_base_patch16_224.sam_in1k', 'vit_base_patch16_224_miil.in21k', 'robust_vit_base_patch16_224' )  

N = 10 # nombre de classes dans le dataset considere

for backbone in backbones:
    
    config.backbone = backbone
    hp_opt = False
    load_architecture(hp_opt, config, N, )
    print('architecture loaded')



architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded
architecture loaded


In [1]:
import timm

vit_models = timm.list_models('*vit_base_patch16_clip_224*', pretrained=True)
for model_name in vit_models:
    print(model_name)

/home/mheuillet/Desktop/robust_training/.venv2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


vit_base_patch16_clip_224.datacompxl
vit_base_patch16_clip_224.dfn2b
vit_base_patch16_clip_224.laion2b
vit_base_patch16_clip_224.laion2b_ft_in1k
vit_base_patch16_clip_224.laion2b_ft_in12k
vit_base_patch16_clip_224.laion2b_ft_in12k_in1k
vit_base_patch16_clip_224.laion400m_e32
vit_base_patch16_clip_224.metaclip_2pt5b
vit_base_patch16_clip_224.metaclip_400m
vit_base_patch16_clip_224.openai
vit_base_patch16_clip_224.openai_ft_in1k
vit_base_patch16_clip_224.openai_ft_in12k
vit_base_patch16_clip_224.openai_ft_in12k_in1k


In [2]:
from timm import create_model
backbone = 'timm/vit_base_patch16_clip_224.laion400m_e32'
model = create_model(backbone, pretrained=True)


In [ ]:




from timm.models import create_model
models = [

    # "swinv2_base_window12to16_192to256.ms_in22k_ft_in1k", #1024
    # "swinv2_cr_small_224.sw_in1k", #768
    # "swinv2_cr_tiny_ns_224.sw_in1k", #768
    # "swinv2_large_window12to16_192to256.ms_in22k_ft_in1k", #1536
    # "vit_base_patch16_224.augreg_in1k",
    
    "resnet50.a1_in1k",
    # "resnet50.clip_cc12",
    # "resnet50.clip_openai",
    # "resnet50.fb_swsl_ig1b_ft_in1k" 

]


import torch.nn as nn

# for model_name in models:
model = timm.create_model(models[0], pretrained=True)
model

def change_head(backbone, model, N):

    if "convnext" in backbone or 'swinv2' in backbone: 
        num_features = model.head.fc.in_features
        model.head.fc = nn.Linear(num_features, N)

    elif "resnet" in backbone:
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, N)

    elif "deit" in backbone:
        num_features = model.head.in_features
        model.head = nn.Linear(num_features, N)

    elif "vit" in backbone: 

        if isinstance(model.head, nn.Identity):
            num_features = 768
            model.head = nn.Linear(num_features, N)

        num_features = model.head.in_features
        model.head = nn.Linear(num_features, N)

    return model

import torch

model = timm.create_model('resnet50', pretrained=False)     
state_dict = torch.load( './state_dicts' + '/{}.pt'.format('resnet50_linf_eps4.0') , weights_only=True, map_location='cpu')
model.load_state_dict(state_dict)
model = change_head(models[0], model, 10)
model


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [42]:
import torch
state_dict = torch.load('./state_dicts/resnet50_linf_eps4.0.ckpt', weights_only=False)
# 2. Create a new dict with comprehension that excludes certain keys

# Keys to remove exactly
exact_keys_to_remove = {"module.normalizer.new_std", "module.normalizer.new_mean"}

# Prefixes we want to remove if a key starts with them
prefixes_to_remove = ("module.attacker",)  # tuple of prefixes, you can add more if needed

filtered_state_dict = {
    k: v
    for k, v in state_dict["model"].items()
    # Keep this entry only if:
    # 1) it’s not in exact_keys_to_remove, AND
    # 2) it doesn't start with any of the given prefixes
    if k not in exact_keys_to_remove
    and not any(k.startswith(prefix) for prefix in prefixes_to_remove)
}

ckpt = {k.replace('module.model.', ''): v for k, v in filtered_state_dict.items()}

# model = timm.models.resnest50(pretrained=False)
# model.load_state_dict(ckpt)
import timm

model = timm.create_model("resnet50", pretrained=False)
model.load_state_dict(ckpt)

torch.save(model.state_dict(), "./state_dicts/resnet50_linf_eps4.0.pt")


state_dict = torch.load("./state_dicts/resnet50_linf_eps4.0.pt", weights_only=False)
model = timm.create_model("resnet50", pretrained=False)
model.load_state_dict(state_dict)

In [44]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     